# HDBSCAN without internet

Add data the Dataset  
[hdbscan 0.8.27 whl](https://www.kaggle.com/something4kag/hdbscan0827-whl?select=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl)

install with commands as shown

For install with internet, note there are currently issues:

see https://github.com/scikit-learn-contrib/hdbscan/issues/457

on import get  
ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

what works  
!pip install hdbscan --no-build-isolation --no-binary :all:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cython

In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/hdbscan0827-whl/hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ hdbscan

# HDBSCAN example

Soft Clustering example to confirm import and package works

In [ ]:
import hdbscan

In [ ]:
# ref hdbscan.readthedocs.io/en/latest/soft_clustering.html
# Soft Clustering for HDBSCAN example for the digits dataset from sklearn
from sklearn import datasets
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
#import numpy as np  

In [ ]:
# project the data into two dimensions to visualize it via t-SNE.
digits = datasets.load_digits()
data = digits.data
projection = TSNE().fit_transform(data)
plt.scatter(*projection.T, )

Now we import hdbscan and then cluster in the full 64 dimensional space. It is important to note that, if we wish to use the soft clustering we should use the prediction_data=True option for HDBSCAN. This will ensure we generate the extra data required that will allow soft clustering to work.

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, prediction_data=True).fit(data)
color_palette = sns.color_palette('Paired', 12)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in clusterer.labels_]
cluster_member_colors = [sns.desaturate(x, p) for x, p in
                         zip(cluster_colors, clusterer.probabilities_)]
plt.scatter(*projection.T, s=50, linewidth=0, c=cluster_member_colors, alpha=0.25)

Certainly a number of clusters were found, but the data is fairly noisy in 64 dimensions, so there are a number of points that have been classified as noise. We can generate a soft clustering to get more information about some of these noise points.

To generate a soft clustering for all the points in the original dataset we use the all_points_membership_vectors() function which takes a clusterer object. If we wanted to get soft cluster membership values for a set of new unseen points we could use membership_vector() instead.

The return value is a two-dimensional numpy array. Each point of the input data is assigned a vector of probabilities of being in a cluster. For a first pass we can visualize the data looking at what the most likely cluster was, by coloring according to the argmax of the probability vector (i.e. the cluster for which a given point has the highest probability of being in).


In [ ]:
soft_clusters = hdbscan.all_points_membership_vectors(clusterer)
color_palette = sns.color_palette('Paired', 12)
cluster_colors = [color_palette[np.argmax(x)]
                  for x in soft_clusters]
plt.scatter(*projection.T, s=50, linewidth=0, c=cluster_colors, alpha=0.25)


There is more, but it works!

Now for Optiver...